In [11]:
import numpy as np
from numpy.random import default_rng

from src.dlla.berg import make_mlp
from src.dlla.hw import prepare_traces_dl, dlla_known_p
from src.dlla.wegener import make_mlp_wegener, wegener_p
from src.tools.la import balance
from src.trace_set.transform import reduce_fixed_fixed
from src.tvla.cri import tvla_t_test, rho_test
from src.tvla.tvla import prepare_tvla

rng = default_rng()

In [12]:
def store_results(method: str, p: float):
    file_name = f"results_false_positives.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{p}\n")


while True:
    num_profile = 10000
    num_attack = 2000
    num_sample_points = 1000

    x = np.array([rng.standard_normal(num_sample_points) for _ in range(num_profile)])
    x_att = np.array([rng.standard_normal(num_sample_points) for _ in range(num_attack)])
    y = rng.binomial(8, .5, num_profile)
    y_att = rng.binomial(8, .5, num_attack)

    print("Prepare traces (1/3)", end="\r")
    x9, y9, x9_att, y9_att = prepare_traces_dl(x, y, x_att, y_att)
    print("Prepare traces (2/3)", end="\r")
    (x2b, y2b), (x2b_att, y2b_att) = reduce_fixed_fixed(x9, y9), reduce_fixed_fixed(x9_att, y9_att, balanced=True)
    print("Prepare traces (3/3)", end="\r")
    (x2, y2), (x2_att, y2_att) = reduce_fixed_fixed(x9, y9), reduce_fixed_fixed(x9_att, y9_att)

    tvla_x, tvla_y = prepare_tvla(x_att, y_att)

    num_balanced = len(y2b_att)
    x9_att, y9_att = x9_att[:num_balanced], y9_att[:num_balanced]
    x2_att, y2_att = x2_att[:num_balanced], y2_att[:num_balanced]

    print("Make model (1/2)    ", end="\r")
    mdl_open = make_mlp(x9, y9, progress=False)
    print("Make model (2/2)    ", end="\r")
    mdl_wegener = make_mlp_wegener(x2b, y2b, False)

    print("TVLA                ", end="\r")
    tvla_ps = np.min(tvla_t_test(tvla_x, tvla_y, 3, progress=False), axis=1)
    rho_p = np.min(rho_test(tvla_x, tvla_y, progress=False), axis=1)[0]

    print("Predict             ", end="\r")
    dlla_k_p = dlla_known_p(mdl_open, x9_att, y9_att)
    dlla_wegener_p = wegener_p(mdl_wegener, x2b_att, y2b_att)

    print(f"TVLA t ({tvla_ps}). TVLA rho ({rho_p}). DLLA known key ({dlla_k_p}). DLLA Wegener ({dlla_wegener_p})")

    for order, p in enumerate(tvla_ps):
        if order > 0:
            store_results(f"tvla_t_{order}", p)

    store_results("tvla_rho",  rho_p)
    store_results("dlla_open", dlla_k_p)
    store_results("dlla_wegener", dlla_wegener_p)


TVLA t ([1.         0.01645546 0.02942342 0.23451523]). TVLA rho (0.058284004248169596). DLLA known key (0.6550754010172752). DLLA Wegener (0.3759980692399809)
TVLA t ([1.         0.04860973 0.02367859 0.21363575]). TVLA rho (0.023230785194484405). DLLA known key (0.9030471881394913). DLLA Wegener (0.6269301638202729)
TVLA t ([1.         0.04492925 0.04336006 0.13933654]). TVLA rho (0.04739111820413622). DLLA known key (0.1805527223431644). DLLA Wegener (0.6629291725036476)
TVLA t ([1.         0.03309565 0.00719863 0.10961343]). TVLA rho (0.015390371026916316). DLLA known key (0.27098642009568935). DLLA Wegener (0.102320536903793)
TVLA t ([1.         0.06743051 0.02626326 0.15802831]). TVLA rho (0.03542023249077597). DLLA known key (0.9347641729186903). DLLA Wegener (0.7738419600076252)
TVLA t ([1.         0.02283698 0.02502594 0.15660384]). TVLA rho (0.022465909617976676). DLLA known key (0.3379314335086375). DLLA Wegener (0.3770653276033243)
TVLA t ([1.         0.01661617 0.02353538 

KeyboardInterrupt: 